# Clipping plane visualization of advection solution on periodic tent slab

We show how to convert a tent slab based on a periodic 2D mesh into a 3D mesh for which we can use the webgui's clipping feature.

In [1]:
from netgen.geom2d import unit_square
from ngsolve import Mesh, CoefficientFunction, x, y, exp, L2, H1, GridFunction, TaskManager, Periodic
import ngsolve as ng
from ngsolve.webgui import Draw
from ngstents import TentSlab
from ngstents.conslaw import Advection
from ngstents.utils import SlabConverter
import tentswebgui

In [2]:
def Make2DPeriodicMesh(xint, yint, maxh=.2):
    """
    Generate a recangular mesh which has periodic left-right and top-bottom boundaries (a torus)
    Inputs:
    xint: tuple defining an interval of the x-axis
    yint: tuple defining as interval of the x-axis
    maxh: max diameter for mesh
    """
    from netgen.geom2d import SplineGeometry
    geo = SplineGeometry()
    pnts = [(xint[0],yint[0]),(xint[1],yint[0]),(xint[1],yint[1]),(xint[0],yint[1])]
    pnums = [geo.AppendPoint(*p) for p in pnts]
    lbot = geo.Append ( ["line", pnums[0], pnums[1]], bc="bottom")
    lright = geo.Append ( ["line", pnums[1], pnums[2]], bc="right")
    geo.Append ( ["line", pnums[0], pnums[3]], leftdomain=0, rightdomain=1, bc="left", copy=lright)
    geo.Append ( ["line", pnums[3], pnums[2]], leftdomain=0, rightdomain=1, bc="top", copy=lbot)
    return geo.GenerateMesh(maxh=maxh)

In [3]:
def MakeTentSlab(mesh, dt):
    heapsize = 10*1000*1000
    wavespeed = 2
    # using causality constant
    local_ctau = True
    global_ctau = 1
    ts = TentSlab(mesh, method="edge", heapsize=heapsize)
    ts.SetMaxWavespeed(wavespeed)
    ts.PitchTents(dt=dt, local_ct=local_ctau, global_ct=global_ctau)
    print("max slope", ts.MaxSlope())
    print("n tents", ts.GetNTents())
    return ts

### Create and visualize the periodic tent slab

In [4]:
maxh = 0.1
dt = 0.2
mesh = Mesh(Make2DPeriodicMesh([0,1], [0,1], maxh=maxh))
ts = MakeTentSlab(mesh, dt)
tentswebgui.Draw(ts) # select "display by tents" to see periodic tents

max slope 0.5000000000000002
n tents 672


NGSTentsWebGuiWidget(value={'ntents': 672, 'nlayers': 27, 'ngsolve_version': '6.2.2103-39-gd1df1e2f3', 'tent_e…

### Construct and setup the Advection instance

In [5]:
order = 4
dt = 0.2
tend = 10*dt

V = L2(mesh, order=order)
u = GridFunction(V)
cl = Advection(u, ts)
flux = (1,0.25)
cl.SetVectorField( CoefficientFunction(flux) )
cl.SetTentSolver("SAT",stages=order+1, substeps=2*order)

pos = (0.5,0.5)
u0 = CoefficientFunction( exp(-100* ((x-pos[0])*(x-pos[0])+(y-pos[1])*(y-pos[1])) ))
cl.SetInitial(u0)

### Convert the tent slab into a periodic 3D mesh and define a space and GridFunction on it

In [6]:
# order_3d must be 1 or 2
order_3d = 2
sc = SlabConverter(ts, p_hd=order_3d)
sc.Convert(tscale=5)

# Define an H1 space on the 3D mesh and get its GridFunction.
V3d = Periodic(H1(sc.mesh, order=order_3d))
u3d = GridFunction(V3d, "u3d")

# Pass the index map generated by SlabConverter to the advection conservation law
cl.SetIdx3d(sc.gfixmap)

# Store vectors for time steps which we want to examine the solution using clipping
# vecs = {1: u3d.vec.CreateVector(), 10: u3d.vec.CreateVector()}
vecs = {1: u3d.vec.CreateVector(), 2: u3d.vec.CreateVector(),
        3: u3d.vec.CreateVector(), 4: u3d.vec.CreateVector(),
        5: u3d.vec.CreateVector(), 6: u3d.vec.CreateVector(),
        7: u3d.vec.CreateVector(), 8: u3d.vec.CreateVector(),
        9: u3d.vec.CreateVector(), 10: u3d.vec.CreateVector()}

for v in vecs.values():
    v[:] = 0

add vertices 0.01906
add volume elements 0.04830
add surface elements 0.06014
Handle periodicity 0.00026
make ngsolve mesh 0.00681
make index map 0.08280277252197266
932 verts, 4022 vol elems, 1576 surf elems in 0.21772.


### Run the advection simulation, saving selected vectors

In [7]:
scene = Draw(u)
t = 0
n = 1
with TaskManager():
    while t < tend-dt/2:
        if n in vecs:
            # While propagating the solution, update the provided GridFunction vector.
            u3d.vec.data = vecs[n]
            cl.Propagate(hdgf=u3d)    # Propagate updates GridFunction u3d 
            vecs[n].data = u3d.vec    # Store updated solution vector
        else:
            cl.Propagate()
        t += dt
        n += 1
        scene.Redraw()

WebGuiWidget(value={'ngsolve_version': '6.2.2103-39-gd1df1e2f3', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, 'd…

## Now we can visualize the solution inside the slab by clipping the 3D mesh

In [8]:
u3d.vec.data = vecs[8]
clip = {'vec': [0, 0, -1], 'dist': 0.01} 
Draw(u3d, clipping=clip, min=0, max=1)

WebGuiWidget(value={'ngsolve_version': '6.2.2103-39-gd1df1e2f3', 'mesh_dim': 3, 'order2d': 2, 'order3d': 2, 'd…

BaseWebGuiScene

### Verify that the periodic 3D mesh is correct
We can save the 3D mesh and check it in Netgen to verify that its boundary regions are correctly defined.  We can also solve a Poisson problem on it to see the periodicity of its boundary

In [10]:
V = ng.Periodic(H1(sc.mesh, order=3, dirichlet='final'))
u, v = V.TnT()
gfu = GridFunction(V)
a = ng.BilinearForm(V)
f = ng.LinearForm(V)
a += ng.grad(u)*ng.grad(v)*ng.dx
f += (x+y)*v*ng.dx
a.Assemble()
f.Assemble()
gfu.vec.data = a.mat.Inverse(freedofs=V.FreeDofs())*f.vec
clip = {'vec': [0, 0, -1], 'dist': 0.01} 
Draw(gfu, clipping=clip)

WebGuiWidget(value={'ngsolve_version': '6.2.2103-39-gd1df1e2f3', 'mesh_dim': 3, 'order2d': 2, 'order3d': 2, 'd…

BaseWebGuiScene

In [11]:
sc.mesh.GetBoundaries()

('bottom', 'right', 'left', 'top', 'base', 'final')